### 0. Imports

In [203]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Vigilar progreso bucles
# -----------------------------------------------------------------------
from tqdm import tqdm

# Gestionar los warnings
# -----------------------------------------------------------------------
import warnings

# modificar el path
# -----------------------------------------------------------------------
import sys
sys.path.append("..")

# importar funciones de soporte
# -----------------------------------------------------------------------
import src.soporte_eda as se
import src.soporte_preprocesamiento as sp

# evaluar objetos literales
# -----------------------------------------------------------------------
from ast import literal_eval 

# statistics functions
# -----------------------------------------------------------------------
from scipy.stats import pearsonr, spearmanr, pointbiserialr


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Introduction - Employee Retention

# 2. Preliminary analysis and data cleaning

## 2.1 Import data

We import the main dataframe first.

In [204]:
general_data_df = pd.read_csv("../data/general_data.csv")
general_data_df.columns = [col.lower() for col in general_data_df.columns]
general_data_df.head(2)

,age,attrition,businesstravel,department,distancefromhome,education,educationfield,employeecount,employeeid,gender,...,numcompaniesworked,over18,percentsalaryhike,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4


Then, the secondary ones. 

The emplyee survey:

In [205]:
employee_survey_df = pd.read_csv("../data/employee_survey_data.csv")
employee_survey_df.columns = [col.lower() for col in employee_survey_df.columns]
employee_survey_df.head(2)

,employeeid,environmentsatisfaction,jobsatisfaction,worklifebalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0


And the manager survey.

In [206]:
manager_survey_df = pd.read_csv("../data/manager_survey_data.csv")
manager_survey_df.columns = [col.lower() for col in manager_survey_df.columns]
manager_survey_df.head(2)

,employeeid,jobinvolvement,performancerating
0,1,3,3
1,2,2,4


## 2.2 Join dataframes

Merge to join the three datasats into one.

In [207]:
employee_attrition = general_data_df.merge(employee_survey_df, how="inner").merge(manager_survey_df, how="inner")

## 2.3 Explore dataframe

Let's perform a preliminary exploration to know what cleaning might be necessary.

In [208]:
se.exploracion_dataframe(employee_attrition)

El número de datos es 4410 y el de columnas es 29

 ..................... 

Las primeras filas del dataframe son:


,age,attrition,businesstravel,department,distancefromhome,education,educationfield,employeecount,employeeid,gender,...,totalworkingyears,trainingtimeslastyear,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,environmentsatisfaction,jobsatisfaction,worklifebalance,jobinvolvement,performancerating
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4



 ..................... 

Los tipos de las columnas y sus valores únicos son:


,tipo_dato,conteo
age,int64,43
attrition,object,2
businesstravel,object,3
department,object,3
distancefromhome,int64,29
education,int64,5
educationfield,object,6
employeecount,int64,1
employeeid,int64,4410
gender,object,2



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


...

,count,pct
worklifebalance,,
3.0,2660,60.3
2.0,1019,23.1
4.0,454,10.3
1.0,239,5.4


La columna JOBINVOLVEMENT tiene 4 valores únicos, de los cuales los primeros son:


,count,pct
jobinvolvement,,
3,2604,59.0
2,1125,25.5
4,432,9.8
1,249,5.6


La columna PERFORMANCERATING tiene 2 valores únicos, de los cuales los primeros son:


,count,pct
performancerating,,
3,3732,84.6
4,678,15.4





**Duplicates**
- There are **0** row level duplicates in the dataset.

Inspecting if there are duplicates by id:


In [209]:
general_data_df.duplicated("employeeid").sum()

0

Inspecting if there are duplicated by all other than the id column:

In [210]:
duplicated = general_data_df.drop(columns="employeeid").duplicated().sum()

duplicated_pct = duplicated / general_data_df.shape[0]

print(f"There are {duplicated} duplicated values, which represent {duplicated_pct:.2f} in proportion")

There are 2912 duplicated values, which represent 0.66 in proportion


An astonishing amount of records are duplicated according to the above check. This is surprising as it is not possible that 66% percent of employees are a duplicate of others...

Let's check their values and ids:

In [211]:
general_data_df.sort_values(by=list(general_data_df.drop(columns=["employeeid"]).columns)).head(6)

,age,attrition,businesstravel,department,distancefromhome,education,educationfield,employeecount,employeeid,gender,...,numcompaniesworked,over18,percentsalaryhike,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager
714,18,No,Non-Travel,Research & Development,1,4,Medical,1,715,Male,...,1.0,Y,22,8,1,0.0,2,0,0,0
2184,18,No,Non-Travel,Research & Development,1,4,Medical,1,2185,Male,...,1.0,Y,22,8,1,0.0,2,0,0,0
3654,18,No,Non-Travel,Research & Development,1,4,Medical,1,3655,Male,...,1.0,Y,22,8,1,0.0,2,0,0,0
1053,18,No,Non-Travel,Research & Development,2,3,Life Sciences,1,1054,Male,...,1.0,Y,24,8,2,0.0,4,0,0,0
2523,18,No,Non-Travel,Research & Development,2,3,Life Sciences,1,2524,Male,...,1.0,Y,24,8,2,0.0,4,0,0,0
3993,18,No,Non-Travel,Research & Development,2,3,Life Sciences,1,3994,Male,...,1.0,Y,24,8,2,0.0,4,0,0,0


Regardless of their employeeid, it is just not possible that there are three youngsters of the same age, on the same department, that live at the same distance from work, that have the same degree of education, make the same, etc. And besides it happens all over the dataset.

In this case it is clear, these records are duplicated and need to be dropped.

Drop the employee_id column and drop duplicates.

In [212]:
employee_attrition.drop(columns="employeeid", inplace=True)
employee_attrition.drop_duplicates(inplace=True)

### 2.3.1 Repeat exploration

In [213]:
se.exploracion_dataframe(employee_attrition)

El número de datos es 1573 y el de columnas es 25

 ..................... 

Las primeras filas del dataframe son:


,age,attrition,businesstravel,department,distancefromhome,education,educationfield,gender,joblevel,jobrole,...,totalworkingyears,trainingtimeslastyear,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,environmentsatisfaction,jobsatisfaction,worklifebalance,jobinvolvement,performancerating
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,1,Research Scientist,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4



 ..................... 

Los tipos de las columnas y sus valores únicos son:


,tipo_dato,conteo
age,int64,43
attrition,object,2
businesstravel,object,3
department,object,3
distancefromhome,int64,29
education,object,5
educationfield,object,6
gender,object,2
joblevel,object,5
jobrole,object,9



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


...

,count,pct
worklifebalance,,
3.0,942,59.9
2.0,357,22.7
4.0,156,9.9
1.0,83,5.3


La columna JOBINVOLVEMENT tiene 4 valores únicos, de los cuales los primeros son:


,count,pct
jobinvolvement,,
3,933,59.3
2,395,25.1
4,155,9.9
1,90,5.7


La columna PERFORMANCERATING tiene 2 valores únicos, de los cuales los primeros son:


,count,pct
performancerating,,
3,1333,84.7
4,240,15.3


**Missing values**

There are some missing values with moderately low impact:

- numcompaniesworked 1.20%
- totalworkingyears	0.57%
- environmentsatisfaction 1.46%
- jobsatisfaction - 1.14%
- worklifebalance - 2.22%

These will be have to be imputed, although a simple imputer might suffice given their low percetanges might not have that much of an impact.

**Low variability columns**

- After the exploration script dropped unique value columns, no other column has low variability of categories or values.


### Categorical columns values

No major problems are observed from either high cardinality, data type errors or typos. There is however, an odd value identified from the report for the variable 'numcompaniesworked'; numcompaniesworked = 0.

It is possible that people have worked in no other company and maybe 0 in this variable accounts for '0 companies prior to this one', although that contradicts the feature definition from the provided column dictionary. However, just to check, the way to check if people come from other companies appart from this same column is to check if the difference between 'totalworkingyears' and 'yearsatcompany' is bigger than 0.


In [214]:
working_years_diff = (employee_attrition["totalworkingyears"] - employee_attrition["yearsatcompany"]) > 0

employee_attrition[working_years_diff & (employee_attrition["numcompaniesworked"] == 0)].shape[0]

211

Almost all records (211/213) that have 'numworkingyears' == 0 should have previous experience. This indicates that 0 could also be an encoding for NaN values, so this values are to be set to NaN. 

In [215]:
employee_attrition.loc[(employee_attrition["numcompaniesworked"] == 0),"numcompaniesworked"] = np.nan

### Numerical columns

For numerical columns, the quickest way to assess value ranges and other statistics is through a descriptive summary:

In [216]:
display(employee_attrition.describe().T)
employee_attrition.select_dtypes(np.number).nunique().reset_index()

,count,mean,std,min,25%,50%,75%,max
age,1573.0,36.904005,9.105911,18.0,30.0,36.0,43.0,60.0
distancefromhome,1573.0,9.158296,8.124414,1.0,2.0,7.0,14.0,29.0
monthlyincome,1573.0,64979.459631,47121.739301,10090.0,28990.0,49000.0,83800.0,199990.0
totalworkingyears,1564.0,11.245524,7.748763,0.0,6.0,10.0,15.0,40.0
yearsatcompany,1573.0,6.970757,6.068348,0.0,3.0,5.0,9.0,40.0
yearssincelastpromotion,1573.0,2.184361,3.203105,0.0,0.0,1.0,3.0,15.0
yearswithcurrmanager,1573.0,4.102988,3.572701,0.0,2.0,3.0,7.0,17.0


,index,0
0,age,43
1,distancefromhome,29
2,monthlyincome,1349
3,totalworkingyears,40
4,yearsatcompany,37
5,yearssincelastpromotion,16
6,yearswithcurrmanager,18


No presence of outliers is detected judging from the ranges of features.



Yearswithcurrmanager and yearssincelastpromotion could also be treated as object.


In [217]:
employee_attrition[["yearssincelastpromotion","yearswithcurrmanager"]] = employee_attrition[["yearssincelastpromotion","yearswithcurrmanager"]].astype("object")

# 4. Export

Export the cleaned data in parquet format to retain data types and compress the information. This data will be used in the next phase of the project `notebooks\2_EDA.ipynb`.

In [218]:
employee_attrition.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1573 entries, 0 to 4409
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      1573 non-null   int64  
 1   attrition                1573 non-null   object 
 2   businesstravel           1573 non-null   object 
 3   department               1573 non-null   object 
 4   distancefromhome         1573 non-null   int64  
 5   education                1573 non-null   object 
 6   educationfield           1573 non-null   object 
 7   gender                   1573 non-null   object 
 8   joblevel                 1573 non-null   object 
 9   jobrole                  1573 non-null   object 
 10  maritalstatus            1573 non-null   object 
 11  monthlyincome            1573 non-null   int64  
 12  numcompaniesworked       1341 non-null   object 
 13  percentsalaryhike        1573 non-null   object 
 14  stockoptionlevel         1573

In [219]:
employee_attrition.to_parquet("../data/cleaned/employee_attrition_clean.parquet")
employee_attrition.to_pickle("../data/cleaned/employee_attrition_clean.pkl")